In [1]:
import get_data
import numpy as np
import pandas as pd
import requests 
import json
import psycopg2 
from sqlalchemy import create_engine
import os

## Import Environmental Variables

In [2]:
propublica_token = os.environ['propublica_token']

In [3]:
postgres_password = os.environ['POSTGRES_PASSWORD']

## Get the data

In [4]:
members_vv, cvote_vv, memvotes_vv = get_data.get_voteview()

In [5]:
members_vv
#Confirm it worked

,congress,chamber,icpsr,state_icpsr,district_code,state_abbrev,party_code,occupancy,last_means,bioname,...,died,nominate_dim1,nominate_dim2,nominate_log_likelihood,nominate_geo_mean_probability,nominate_number_of_votes,nominate_number_of_errors,conditional,nokken_poole_dim1,nokken_poole_dim2
0,117,President,99912,99,0,USA,200,0.0,0.0,"TRUMP, Donald John",...,NaN,0.403,0.162,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,117,President,99913,99,0,USA,100,0.0,0.0,"BIDEN, Joseph Robinette, Jr.",...,NaN,-0.320,0.283,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,117,House,20301,41,3,AL,200,NaN,NaN,"ROGERS, Mike Dennis",...,NaN,0.360,0.463,-74.23598,0.89924,699.0,24.0,NaN,0.422,0.539
3,117,House,21102,41,7,AL,100,NaN,NaN,"SEWELL, Terri",...,NaN,-0.393,0.402,-22.40082,0.96907,713.0,9.0,NaN,-0.498,0.489
4,117,House,21193,41,5,AL,200,NaN,NaN,"BROOKS, Mo",...,NaN,0.646,-0.404,-141.85497,0.81727,703.0,66.0,NaN,0.684,-0.270
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
550,117,Senate,40915,56,0,WV,100,NaN,NaN,"MANCHIN, Joe, III",...,NaN,-0.059,0.415,-71.99850,0.91095,772.0,21.0,NaN,-0.074,0.349
551,117,Senate,29940,25,0,WI,100,NaN,NaN,"BALDWIN, Tammy",...,NaN,-0.493,-0.136,-21.62235,0.97379,814.0,10.0,NaN,-0.474,-0.006
552,117,Senate,41111,25,0,WI,200,NaN,NaN,"JOHNSON, Ron",...,NaN,0.620,-0.171,-126.51787,0.84793,767.0,54.0,NaN,0.602,0.046
553,117,Senate,20953,68,0,WY,200,NaN,NaN,"LUMMIS, Cynthia M.",...,NaN,0.690,-0.358,-123.41203,0.85191,770.0,53.0,NaN,0.720,-0.342


In [6]:
#get user agent and confirm it works
useragent = get_data.get_useragent()
useragent

'python-requests/2.28.1'

In [7]:
#get propublica data and confirm it works
members_pp = get_data.get_propublica(propublica_token, useragent, email = 'brc4cb@virginia.edu')
members_pp

/usr/local/lib/python3.11/site-packages/IPython/lib/pretty.py:778: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  output = repr(obj)
/usr/local/lib/python3.11/site-packages/IPython/core/formatters.py:342: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  return method()


,id,title,short_title,api_uri,first_name,middle_name,last_name,suffix,date_of_birth,gender,...,state,district,at_large,geoid,missed_votes_pct,votes_with_party_pct,votes_against_party_pct,senate_class,state_rank,lis_id
0,A000370,Representative,Rep.,https://api.propublica.org/congress/v1/members...,Alma,None,Adams,None,1946-05-27,F,...,NC,12,False,3712,0.32,99.02,0.87,NaN,NaN,NaN
1,A000055,Representative,Rep.,https://api.propublica.org/congress/v1/members...,Robert,B.,Aderholt,None,1965-07-22,M,...,AL,4,False,0104,1.51,96.24,3.65,NaN,NaN,NaN
2,A000371,Representative,Rep.,https://api.propublica.org/congress/v1/members...,Pete,None,Aguilar,None,1979-06-19,M,...,CA,31,False,0631,0.32,98.80,1.09,NaN,NaN,NaN
3,A000372,Representative,Rep.,https://api.propublica.org/congress/v1/members...,Rick,None,Allen,None,1951-11-07,M,...,GA,12,False,1312,3.03,90.92,8.97,NaN,NaN,NaN
4,A000376,Representative,Rep.,https://api.propublica.org/congress/v1/members...,Colin,None,Allred,None,1983-04-15,M,...,TX,32,False,4832,1.08,98.14,1.75,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
551,W000817,"Senator, 1st Class",Sen.,https://api.propublica.org/congress/v1/members...,Elizabeth,None,Warren,None,1949-06-22,F,...,MA,NaN,NaN,NaN,0.69,97.79,2.21,1,senior,S366
552,W000802,"Senator, 1st Class",Sen.,https://api.propublica.org/congress/v1/members...,Sheldon,None,Whitehouse,None,1955-10-20,M,...,RI,NaN,NaN,NaN,1.39,99.30,0.70,1,junior,S316
553,W000437,"Senator, 1st Class",Sen.,https://api.propublica.org/congress/v1/members...,Roger,None,Wicker,None,1951-07-05,M,...,MS,NaN,NaN,NaN,1.39,89.73,10.27,1,senior,S318
554,W000779,"Senator, 3rd Class",Sen.,https://api.propublica.org/congress/v1/members...,Ron,None,Wyden,None,1949-05-03,M,...,OR,NaN,NaN,NaN,0.92,99.07,0.93,3,senior,S247


In [8]:
#merge two dataframes and confirm it works
members = get_data.merge_members(members_pp= members_pp, members_vv = members_vv)
members

,title,short_title,first_name,middle_name,last_name,suffix,congress,chamber,icpsr,state,...,office,phone,fax,missed_votes_pct,votes_with_party_pct,votes_against_party_pct,DWNOMINATE,propublica_id,propublica_endpoint,last_updated
0,Representative,Rep.,Alma,None,ADAMS,None,117.0,House,21545.0,NC,...,2436 Rayburn House Office Building,202-225-1510,None,0.32,99.02,0.87,-0.465,A000370,https://api.propublica.org/congress/v1/members...,2022-11-03 17:30:12 -0400
1,Representative,Rep.,Robert,B.,ADERHOLT,None,117.0,House,29701.0,AL,...,266 Cannon House Office Building,202-225-4876,None,1.51,96.24,3.65,0.380,A000055,https://api.propublica.org/congress/v1/members...,2022-11-03 17:30:10 -0400
2,Representative,Rep.,Pete,None,AGUILAR,None,117.0,House,21506.0,CA,...,109 Cannon House Office Building,202-225-3201,None,0.32,98.80,1.09,-0.296,A000371,https://api.propublica.org/congress/v1/members...,2022-11-03 17:30:12 -0400
3,Representative,Rep.,Rick,None,ALLEN,None,117.0,House,21516.0,GA,...,570 Cannon House Office Building,202-225-2823,None,3.03,90.92,8.97,0.699,A000372,https://api.propublica.org/congress/v1/members...,2022-11-03 17:30:10 -0400
4,Representative,Rep.,Colin,None,ALLRED,None,117.0,House,21900.0,TX,...,114 Cannon House Office Building,202-225-2231,None,1.08,98.14,1.75,-0.432,A000376,https://api.propublica.org/congress/v1/members...,2022-11-03 17:30:11 -0400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
551,"Senator, 1st Class",Sen.,Elizabeth,None,WARREN,None,117.0,Senate,41301.0,MA,...,309 Hart Senate Office Building,202-224-4543,None,0.69,97.79,2.21,-0.753,W000817,https://api.propublica.org/congress/v1/members...,2022-10-12 09:45:23 -0400
552,"Senator, 1st Class",Sen.,Sheldon,None,WHITEHOUSE,None,117.0,Senate,40704.0,RI,...,530 Hart Senate Office Building,202-224-2921,None,1.39,99.30,0.70,-0.354,W000802,https://api.propublica.org/congress/v1/members...,2022-10-01 01:45:44 -0400
553,"Senator, 1st Class",Sen.,Roger,None,WICKER,None,117.0,Senate,29534.0,MS,...,555 Dirksen Senate Office Building,202-224-6253,None,1.39,89.73,10.27,0.377,W000437,https://api.propublica.org/congress/v1/members...,2022-10-01 01:45:44 -0400
554,"Senator, 3rd Class",Sen.,Ron,None,WYDEN,None,117.0,Senate,14871.0,OR,...,221 Dirksen Senate Office Building,202-224-5244,202-228-2717,0.92,99.07,0.93,-0.330,W000779,https://api.propublica.org/congress/v1/members...,2022-10-01 01:45:44 -0400


## Get data into first normal form

In [9]:
members.head(1).T
#Options for primary key - ICPSR or propublica_id
#nothing non-atomic or no repeating groups

,0
title,Representative
short_title,Rep.
first_name,Alma
middle_name,None
last_name,ADAMS
suffix,None
congress,117.0
chamber,House
icpsr,21545.0
state,NC


In [10]:
members['propublica_id'].isna().value_counts()
#there are no missing propublica_ids so we can use it 

False    556
Name: propublica_id, dtype: int64

In [11]:
cvote_vv.head(1).T
#nothing non-atomic, and no repeating groups
#primary key is chamber and rollnumber together- unique id number for votes in that chamber of congress

,0
congress,117
chamber,House
rollnumber,1
date,2021-01-03
session,1
clerk_rollnumber,2
yea_count,216
nay_count,211
nominate_mid_1,-0.057
nominate_mid_2,0.038


In [12]:
memvotes_vv.head(1).T
#Primary key is icpsr and rollnumber, no non atomic or repeating groups

,0
congress,117
chamber,House
rollnumber,1
icpsr,14066.0
cast_code,6
prob,100.0


## Second normal form
- No partial dependencies - only happens if you have a superkey

In [13]:
#Good on members table because it is a single column
# For chamber and rollnumber we are good
#For memvotes we are also good 

## Third normal form
- No transitive dependencies 

In [14]:
# we are good with members- all elements are attributes of person instead of something else

## Initializing the Database in Postgres

In [15]:
server = psycopg2.connect(
    user = 'postgres',
    password = postgres_password,
    host = 'postgres',
    port = '5432')
server.autocommit = True

In [16]:
cursor = server.cursor()

In [17]:
try: 
    cursor.execute("CREATE DATABASE contrans")
except:
    cursor.execute("DROP DATABASE contrans")
    cursor.execute("CREATE DATABASE contrans")

## Add the data to contrans DB

In [18]:
engine = create_engine("postgresql+psycopg2://{user}:{password}@{host}:{port}/{db}".format(
    user = 'postgres', 
    password = postgres_password, 
    host = 'postgres', 
    port = '5432', 
    db= 'contrans'))

In [19]:
members.to_sql('members', con = engine, if_exists = 'replace', index=False, chunksize=1000)
memvotes_vv.to_sql('member_vote', con = engine, if_exists = 'replace', index=False, chunksize=1000)
cvote_vv.to_sql('rollcalls', con = engine, if_exists = 'replace', index=False, chunksize=1000)

1723

## Example queries

In [20]:
myquery = """
SELECT * FROM rollcalls
LIMIT 5
"""
pd.read_sql(myquery, con = engine)
#limit pulls the first 5

,congress,chamber,rollnumber,date,session,clerk_rollnumber,yea_count,nay_count,nominate_mid_1,nominate_mid_2,nominate_spread_1,nominate_spread_2,nominate_log_likelihood,bill_number,vote_result,vote_desc,vote_question,dtl_desc
0,117,House,1,2021-01-03,1,2,216,211,-0.057,0.038,0.707,0.274,-4.069,None,Pelosi,None,Election of the Speaker,None
1,117,House,2,2021-01-03,1,3,371,2,0.000,0.000,0.000,0.000,0.000,HRES1,Passed,Authorizing and directing the Speaker to admin...,On Agreeing to the Resolution,None
2,117,House,3,2021-01-04,1,4,214,204,0.019,-0.125,0.711,-0.235,-0.242,HRES8,Passed,Adopting the Rules of the House of Representat...,On Motion to Table the Motion to Postpone to a...,None
3,117,House,4,2021-01-04,1,5,214,196,0.020,-0.133,0.710,-0.240,-0.240,HRES8,Passed,Adopting the Rules of the House of Representat...,Table Motion to Refer,None
4,117,House,5,2021-01-04,1,6,217,204,0.019,-0.124,0.711,-0.234,-0.242,HRES8,Passed,Adopting the Rules of the House of Representat...,On Ordering the Previous Question,None
